# Triggers in MySQL

#### **1. What are Triggers?**
A **trigger** is a set of instructions in MySQL that are automatically executed in response to specific events on a table. Triggers are used to enforce business rules, maintain data integrity, and perform automated tasks.

#### **2. Types of Triggers**
MySQL supports the following types of triggers based on events:
1. **BEFORE INSERT**: Executes before a new record is inserted.
2. **AFTER INSERT**: Executes after a new record is inserted.
3. **BEFORE UPDATE**: Executes before an existing record is updated.
4. **AFTER UPDATE**: Executes after an existing record is updated.
5. **BEFORE DELETE**: Executes before a record is deleted.
6. **AFTER DELETE**: Executes after a record is deleted.

---

#### **3. Syntax of a Trigger**

```sql
CREATE TRIGGER trigger_name
AFTER|BEFORE INSERT|UPDATE|DELETE
ON table_name
FOR EACH ROW
BEGIN
    -- Trigger body (SQL statements)
END;
```

---

#### **4. Creating a Sample Database**
We will create a `bank` database with the following tables:
- **accounts**: To store account details.
- **transactions**: To record transactions.

---

#### **5. Sample Schema**
```sql
-- Create database
CREATE DATABASE bank;

USE bank;

-- Create accounts table
CREATE TABLE accounts (
    account_id INT AUTO_INCREMENT PRIMARY KEY,
    account_holder VARCHAR(100),
    balance DECIMAL(10, 2) DEFAULT 0.00
);

-- Create transactions table
CREATE TABLE transactions (
    transaction_id INT AUTO_INCREMENT PRIMARY KEY,
    account_id INT,
    transaction_type ENUM('credit', 'debit'),
    amount DECIMAL(10, 2),
    transaction_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (account_id) REFERENCES accounts(account_id)
);
```

---

#### **6. Insert Sample Data**
```sql
-- Insert data into accounts
INSERT INTO accounts (account_holder, balance)
VALUES 
('Alice', 5000.00),
('Bob', 3000.00),
('Charlie', 7000.00),
('David', 10000.00),
('Eve', 1200.00),
('Frank', 600.00),
('Grace', 800.00),
('Hannah', 950.00),
('Ian', 400.00),
('Jack', 8000.00),
('Karen', 5500.00),
('Leo', 2000.00),
('Mona', 4800.00),
('Nina', 300.00),
('Oscar', 1000.00),
('Paul', 2300.00),
('Quinn', 1500.00),
('Rita', 2900.00),
('Steve', 8700.00),
('Trudy', 1300.00);

-- Insert data into transactions
INSERT INTO transactions (account_id, transaction_type, amount)
VALUES
(1, 'credit', 1000.00),
(1, 'debit', 500.00),
(2, 'debit', 1000.00),
(2, 'credit', 200.00),
(3, 'credit', 1500.00),
(3, 'debit', 800.00),
(4, 'debit', 2000.00),
(4, 'credit', 500.00),
(5, 'credit', 700.00),
(5, 'debit', 300.00),
(6, 'debit', 100.00),
(6, 'credit', 400.00),
(7, 'credit', 500.00),
(7, 'debit', 200.00),
(8, 'credit', 100.00),
(8, 'debit', 50.00),
(9, 'credit', 300.00),
(9, 'debit', 100.00),
(10, 'credit', 2000.00),
(10, 'debit', 1000.00);
```

---

#### **7. Creating Triggers**

##### **a. BEFORE INSERT Trigger**
Ensure that no transaction is added with a negative amount:
```sql
CREATE TRIGGER before_transaction_insert
BEFORE INSERT ON transactions
FOR EACH ROW
BEGIN
    IF NEW.amount <= 0 THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Transaction amount must be greater than zero.';
    END IF;
END;
```

##### **b. AFTER INSERT Trigger**
Automatically update the account balance after a transaction:
```sql
CREATE TRIGGER after_transaction_insert
AFTER INSERT ON transactions
FOR EACH ROW
BEGIN
    IF NEW.transaction_type = 'credit' THEN
        UPDATE accounts
        SET balance = balance + NEW.amount
        WHERE account_id = NEW.account_id;
    ELSEIF NEW.transaction_type = 'debit' THEN
        UPDATE accounts
        SET balance = balance - NEW.amount
        WHERE account_id = NEW.account_id;
    END IF;
END;
```

##### **c. BEFORE DELETE Trigger**
Prevent deleting a transaction if it will result in an invalid balance:
```sql
CREATE TRIGGER before_transaction_delete
BEFORE DELETE ON transactions
FOR EACH ROW
BEGIN
    DECLARE current_balance DECIMAL(10, 2);
    SELECT balance INTO current_balance FROM accounts WHERE account_id = OLD.account_id;
    
    IF OLD.transaction_type = 'debit' THEN
        SET current_balance = current_balance + OLD.amount;
    ELSEIF OLD.transaction_type = 'credit' THEN
        SET current_balance = current_balance - OLD.amount;
    END IF;
    
    IF current_balance < 0 THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Cannot delete transaction as it will result in a negative balance.';
    END IF;
END;
```

---

#### **8. Test the Triggers**

- **Trigger 1 (BEFORE INSERT)**:
  ```sql
  INSERT INTO transactions (account_id, transaction_type, amount)
  VALUES (1, 'credit', -100); -- This should fail
  ```

- **Trigger 2 (AFTER INSERT)**:
  ```sql
  INSERT INTO transactions (account_id, transaction_type, amount)
  VALUES (1, 'credit', 200.00); -- Alice’s balance should increase to 1200.00
  ```

- **Trigger 3 (BEFORE DELETE)**:
  ```sql
  DELETE FROM transactions WHERE transaction_id = 1; -- This should fail if it causes a negative balance
  ```

---

#### **9. Key Advantages of Triggers**
- **Automation**: Triggers automate repetitive tasks such as updating dependent records.
- **Data Integrity**: Ensure the database remains consistent, e.g., preventing negative balances.
- **Auditing**: Record changes or enforce rules transparently.

---

#### **10. Conclusion**
Triggers in MySQL are a powerful feature for enforcing business logic and maintaining data integrity. They work automatically, ensuring the database state remains consistent without requiring manual intervention.

